In [116]:
import numpy as np
import pandas as pd
from random import uniform, randint, seed
from faker import Faker
seed(1738)

In [117]:
def synth_oakland_data():
    elec = pd.DataFrame()
    disp = pd.DataFrame()
    
    fake = Faker()
    elec['latitude']   = [round(uniform(37.778, 37.807), 3) for _ in xrange(20)]
    elec['longitude']  = [round(uniform(-122.188, -122.228), 3) for _ in xrange(20)]
    elec['temp_names'] = [fake.name() for _ in xrange(20)]
    elec['first_name'] = elec.temp_names.apply(lambda x: x.split()[0])
    elec['last_name']  = elec.temp_names.apply(lambda x: x.split()[1])
    elec['rating']     = [round(float(randint(2, 5)), 1) for _ in xrange(20)]
    elec['available']  = True
    
    disp['latitude']   = [round(uniform(37.778, 37.807), 3) for _ in xrange(3)]
    disp['longitude']  = [round(uniform(-122.188, -122.228), 3) for _ in xrange(3)]
    disp['temp_names'] = [fake.name() for _ in xrange(3)]
    disp['first_name'] = elec.temp_names.apply(lambda x: x.split()[0])
    disp['last_name']  = elec.temp_names.apply(lambda x: x.split()[1])
    
    elec.drop('temp_names', axis=1, inplace=True)
    disp.drop('temp_names', axis=1, inplace=True)
    
    return elec, disp

def _euclidean(a, b):
    xa, ya = a
    xb, yb = b
    dist = ((xa - xb) ** 2 + (ya - yb) ** 2) ** 0.5
    return dist

In [118]:
class SunWatch(object):
    def __init__(self, electricians):
        self.threes = None
        self.fours = None
        self.fives = None
        self.dispatches = None
        self.electricians = electricians

    def queue_dispatches(self, dispatches):
        self.dispatches = dispatches

    def _euclidean(self, a, b):
        xa, ya = a
        xb, yb = b
        dist = ((xa - xb) ** 2 + (ya - yb) ** 2) ** 0.5
        return dist

    def match_dispatch(self):
        DISPATCH = dispatches[['latitude', 'longitude']].ix[0]

        df = self.electricians.copy()
        df['dist'] = [self._euclidean(df[['latitude', 'longitude']].ix[i], DISPATCH) for i in xrange(df.shape[0])]

        self.threes = pd.DataFrame(df[df.rating == 3][df.available].sort_values(by=['dist'])[:3].values)
        self.fours  = pd.DataFrame(df[df.rating == 4][df.available].sort_values(by=['dist'])[:3].values)
        self.fives  = pd.DataFrame(df[df.rating == 5][df.available].sort_values(by=['dist'])[:3].values)

        self.threes.columns = df.columns[:]
        self.fours.columns  = df.columns[:]
        self.fives.columns  = df.columns[:]

        return self.fives.ix[0], self.fours.ix[0], self.threes.ix[0]

    def match_all(self):
        for i in self.dispatches.shape[0]:
            DISPATCH = dispatches[['latitude', 'longitude']].ix[0]

            df = self.electricians.copy()
            df['dist'] = [self._euclidean(df[['latitude', 'longitude']].ix[i], DISPATCH) for i in xrange(df.shape[0])]

            self.threes = pd.DataFrame(df[df.rating == 3][df.available].sort_values(by=['dist'])[:3].values)
            self.fours  = pd.DataFrame(df[df.rating == 4][df.available].sort_values(by=['dist'])[:3].values)
            self.fives  = pd.DataFrame(df[df.rating == 5][df.available].sort_values(by=['dist'])[:3].values)

            return self.fives, fours, threes

In [119]:
electrician, dispatches = synth_oakland_data()

In [120]:
temp = SunWatch(electrician)
temp.queue_dispatches(dispatches)
# temp.match_dispatch()

In [121]:
z = temp.match_dispatch()

shown = pd.DataFrame(columns=temp.electricians.columns[:])
for i in xrange(3):
    shown.loc[0] = z[0]
    shown.loc[1] = z[1]
    shown.loc[2] = z[2]

In [124]:
dispatches.to_csv('dispatches_data.csv')
electrician.to_csv('electrician_data.csv')
shown.to_csv('best_three.csv')